<a href="https://colab.research.google.com/github/philipsfv-wq/JONATHANMANGANO/blob/main/Desafio_Kartado_Ajustado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
from openpyxl import load_workbook
import pandas as pd
from datetime import datetime

# --- Parte 1: Leitura da planilha ---

filename='Planilha_Cliente_Com_Erros.xlsx'

# Ensure the file exists in the Colab environment before attempting to load
try:
    wb = load_workbook(filename=filename)
except FileNotFoundError:
    print(f"Error: The file '{filename}' was not found.")
    print("Please upload the file to your Colab environment.")
    # You can add code here to handle the missing file, e.g., exit or ask the user to upload
    raise # Re-raise the exception so the user sees it

sheet = wb[wb.sheetnames[1]] # Assumindo que a aba de dados é a segunda (Microrrevestimento)
data = []

headers = [cell.value for cell in sheet[1]]

for row in sheet.iter_rows(min_row=2):
    row_dict = {}
    for key, cell in zip(headers, row):
        row_dict[key] = cell.value
    data.append(row_dict)

# Filtro pra utilizar somente as linhas da planilha com dados
data = data[0:23]

# --- Funções de Correção Auxiliares ---

def arredondar_para_float(valor, casas=3):
    """Converte para float e arredonda para N casas decimais."""
    try:
        return round(float(valor), casas)
    except (TypeError, ValueError):
        return valor

def formatar_data(valor):
    """Converte valores numéricos do Excel (44805) ou objetos datetime em objeto datetime."""
    if isinstance(valor, (int, float)):
        return pd.to_datetime('1899-12-30') + pd.to_timedelta(valor, unit='D')
    elif isinstance(valor, datetime):
        return valor
    return valor

# --- Parte 2: Implementação das Correções ---

for i, apontamento in enumerate(data):

    # 1. Comprimento (Inteiro)
    comprimento_ajustado = arredondar_para_float(apontamento.get('Comprimento (m)'), casas=0)
    try:
        # Check if the adjusted value is a number before converting to int
        if isinstance(comprimento_ajustado, (int, float)):
            apontamento['Comprimento (m)'] = int(comprimento_ajustado)
        else:
            apontamento['Comprimento (m)'] = apontamento.get('Comprimento (m)') # Keep original value if not convertible
    except (TypeError, ValueError):
        apontamento['Comprimento (m)'] = apontamento.get('Comprimento (m)')  # Mantém valor original

    # 2. Largura (Float com 3 casas decimais)
    largura_ajustada = arredondar_para_float(apontamento.get('Largura (m)'), casas=3)
    apontamento['Largura (m)'] = largura_ajustada

    # 3. Classe ('Trinc' -> 'Trincas')
    classe = apontamento.get('Classe')
    if classe == 'Trinc':
        apontamento['Classe'] = 'Trincas'

    # 4. Data (ex: '44805' -> data)
    data_encontrado = apontamento.get('Encontrado em')
    if data_encontrado is not None: # Check if the value is not None before formatting
        apontamento['Encontrado em'] = formatar_data(data_encontrado)


    # 5. Sentido ('Nordeste' -> 'Não se Aplica')
    if apontamento.get('Sentido') == 'Nordeste':
         apontamento['Sentido'] = 'Não se Aplica'

# --- Parte 3: Análise ---

dados = {}
for a in data:
    classe = a.get('Classe')
    if classe:
        dados[classe] = dados.get(classe, 0) + 1

print(dados)

{'Microrrevestimento': 18, 'Trincas': 3, 'Panos': 2}


/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [46]:
# Save the DataFrame to an Excel file
output_filename = 'Planilha_Cliente_Corrigida.xlsx'
df.to_excel(output_filename, index=False)

# Provide a download link
from google.colab import files
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
df = pd.DataFrame(data)
display(df.head())

,km,km final,km de Projeto,km final de Projeto,Status,Equipe/Empreiteira,Encontrado em,Executado em,Prazo,Sentido,...,Cód interno,Lote,Comprimento (m),Largura (m),Observações,Recurso_1,Quantidade_1,Recurso_2,Quantidade_2,caminho
0,0.36,0.85,None,None,Identificado,Insttale,2022-09-01,None,2022-12-22,Sul,...,None,Lote 3,490,3.6,MFS+Micro,MFS,=R2*S2,Micro,=R2*S2,None
1,0.36,1.36,None,None,Identificado,Insttale,2022-09-01,None,2022-12-22,Norte,...,None,Lote 3,1000,3.6,MFS+Micro,MFS,=R3*S3,Micro,=R3*S3,None
2,0.64,1.64,None,None,Identificado,Insttale,2022-09-01,None,2022-12-22,Sul,...,None,Lote 3,1000,2.5,Micro,Micro,=R4*S4,None,None,None
3,0.64,1.64,None,None,Identificado,Insttale,2022-09-01,None,2022-12-22,Norte,...,None,Lote 3,1000,2.5,Micro,Micro,=R5*S5,None,None,https://kartadotest.sharepoint.com/:f:/s/Karta...
4,0.86,1.31,None,None,Identificado,Insttale,2022-09-01,None,2022-12-22,Sul,...,None,Lote 3,450,3.6,FS(20%)+MFS+Micro,None,=R6*S6,None,None,https://kartadotest.sharepoint.com/:f:/s/Karta...


# Task
Maintain the data from "Planilha_Cliente_Com_Erros.xlsx", apply the corrections defined in the code, and generate a downloadable Excel file with the formatting of "Template_Kartado_Limpo.xlsx".

## Load data from the source file

### Subtask:
Load the data from `Planilha_Cliente_Com_Erros.xlsx` and apply the necessary corrections as defined in the current code.


**Reasoning**:
The first code cell already contains the logic to load the data and apply the corrections. Executing this cell will perform the required steps of the subtask.



In [48]:
from openpyxl import load_workbook
import pandas as pd
from datetime import datetime

# --- Parte 1: Leitura da planilha ---

filename='Planilha_Cliente_Com_Erros.xlsx'

# Ensure the file exists in the Colab environment before attempting to load
try:
    wb = load_workbook(filename=filename)
except FileNotFoundError:
    print(f"Error: The file '{filename}' was not found.")
    print("Please upload the file to your Colab environment.")
    # You can add code here to handle the missing file, e.g., exit or ask the user to upload
    raise # Re-raise the exception so the user sees it

sheet = wb[wb.sheetnames[1]] # Assumindo que a aba de dados é a segunda (Microrrevestimento)
data = []

headers = [cell.value for cell in sheet[1]]

for row in sheet.iter_rows(min_row=2):
    row_dict = {}
    for key, cell in zip(headers, row):
        row_dict[key] = cell.value
    data.append(row_dict)

# Filtro pra utilizar somente as linhas da planilha com dados
data = data[0:23]

# --- Funções de Correção Auxiliares ---

def arredondar_para_float(valor, casas=3):
    """Converte para float e arredonda para N casas decimais."""
    try:
        return round(float(valor), casas)
    except (TypeError, ValueError):
        return valor

def formatar_data(valor):
    """Converte valores numéricos do Excel (44805) ou objetos datetime em objeto datetime."""
    if isinstance(valor, (int, float)):
        return pd.to_datetime('1899-12-30') + pd.to_timedelta(valor, unit='D')
    elif isinstance(valor, datetime):
        return valor
    return valor

# --- Parte 2: Implementação das Correções ---

for i, apontamento in enumerate(data):

    # 1. Comprimento (Inteiro)
    comprimento_ajustado = arredondar_para_float(apontamento.get('Comprimento (m)'), casas=0)
    try:
        # Check if the adjusted value is a number before converting to int
        if isinstance(comprimento_ajustado, (int, float)):
            apontamento['Comprimento (m)'] = int(comprimento_ajustado)
        else:
            apontamento['Comprimento (m)'] = apontamento.get('Comprimento (m)') # Keep original value if not convertible
    except (TypeError, ValueError):
        apontamento['Comprimento (m)'] = apontamento.get('Comprimento (m)')  # Mantém valor original

    # 2. Largura (Float com 3 casas decimais)
    largura_ajustada = arredondar_para_float(apontamento.get('Largura (m)'), casas=3)
    apontamento['Largura (m)'] = largura_ajustada

    # 3. Classe ('Trinc' -> 'Trincas')
    classe = apontamento.get('Classe')
    if classe == 'Trinc':
        apontamento['Classe'] = 'Trincas'

    # 4. Data (ex: '44805' -> data)
    data_encontrado = apontamento.get('Encontrado em')
    if data_encontrado is not None: # Check if the value is not None before formatting
        apontamento['Encontrado em'] = formatar_data(data_encontrado)


    # 5. Sentido ('Nordeste' -> 'Não se Aplica')
    if apontamento.get('Sentido') == 'Nordeste':
         apontamento['Sentido'] = 'Não se Aplica'

# --- Parte 3: Análise ---

dados = {}
for a in data:
    classe = a.get('Classe')
    if classe:
        dados[classe] = dados.get(classe, 0) + 1

print(dados)

{'Microrrevestimento': 18, 'Trincas': 3, 'Panos': 2}


/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## Load the template file

### Subtask:
Load the `Template_Kartado_Limpo.xlsx` file using `openpyxl` to preserve its formatting.


**Reasoning**:
Load the template Excel file using openpyxl to preserve its formatting.



In [49]:
template_filename = 'Template_Kartado_Limpo.xlsx'

try:
    template_wb = load_workbook(filename=template_filename)
except FileNotFoundError:
    print(f"Error: The file '{template_filename}' was not found.")
    print("Please upload the file to your Colab environment.")
    raise # Re-raise the exception so the user sees it

template_sheet = template_wb.active

## Prepare data for writing

### Subtask:
Ensure the processed data from the source file is in a format suitable for writing into the template, aligning with the template's column structure.


**Reasoning**:
Create a new list to store the data in the order of the template columns and iterate through each dictionary in the `data` list to create a new list containing the values in the order that matches the column order of the `template_sheet`, then append this newly created list of ordered values to the list created in step 1.



In [50]:
ordered_data = []
template_headers = [cell.value for cell in template_sheet[1]]

for apontamento in data:
    ordered_row = []
    for header in template_headers:
        # Get the value from the source data dictionary using the template header
        # If the header doesn't exist in the source data, use None
        ordered_row.append(apontamento.get(header, None))
    ordered_data.append(ordered_row)

# You can display the first few rows of ordered_data to verify
# print(ordered_data[:5])

## Write data into the template

### Subtask:
Iterate through the processed data and write it into the appropriate rows of the loaded template, starting from the second row (assuming the first row contains headers).


**Reasoning**:
Iterate through the processed data and write it into the appropriate rows of the loaded template, starting from the second row.



In [51]:
# Iterate through ordered_data and write to template_sheet
for row_index, row_data in enumerate(ordered_data):
    # openpyxl uses 1-based indexing, and we start writing from the second row
    current_row = row_index + 2
    for col_index, cell_value in enumerate(row_data):
        # openpyxl uses 1-based indexing for columns as well
        template_sheet.cell(row=current_row, column=col_index + 1).value = cell_value

## Save the new file

### Subtask:
Save the modified template with the new data to a new Excel file.


**Reasoning**:
Save the modified template workbook to a new Excel file.



In [52]:
# Define the name of the output file
output_filename = 'Planilha_Cliente_Corrigida.xlsx'

# Save the workbook
template_wb.save(output_filename)

## Provide download link

### Subtask:
Generate a download link for the newly created file.


**Reasoning**:
Import the necessary library and generate the download link for the output file.



In [53]:
from google.colab import files
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Summary:

### Data Analysis Key Findings

*   The data from `Planilha_Cliente_Com_Erros.xlsx` was successfully loaded and corrected.
*   Corrections included rounding 'Comprimento (m)' to integers, 'Largura (m)' to 3 decimal places, replacing 'Trinc' with 'Trincas' in the 'Classe' column, converting dates, and changing 'Nordeste' to 'Não se Aplica' in the 'Sentido' column.
*   The template file `Template_Kartado_Limpo.xlsx` was loaded, preserving its formatting.
*   The corrected data was successfully written into the template file, aligning with the template's column structure.
*   The final corrected data, formatted according to the template, was saved as `Planilha_Cliente_Corrigida.xlsx`.

### Insights or Next Steps

*   The process effectively cleaned and standardized the client data according to predefined rules and formatted it using a template, making it ready for further use or analysis.
*   Consider adding validation steps to ensure the corrected data conforms to specific data types or ranges, potentially logging any values that could not be corrected as expected.
